# Preprocesamiento del set de datos

------------------------------------------

En esta sección se presenta el preprocesamiento de datos realizado al dataset diabetes.csv

## Librerias y configuraciones iniciales.


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
np.random.seed(3)

## Lectura del dataset

-------------------------

Se lee el set de datos crudo para comenzar a realizar el preprocesamiento.


In [2]:
df = pd.read_csv('https://github.com/cris21309/tdsp_template_grupo8/blob/master/scripts/data_acquisition/diabetes.csv?raw=true')
df.head(5)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


## Limpieza del dataset

--------------------------------------

Se inicia identificando los registros duplicados del dataset, para esto se utiliza el metodo dupplicated de pandas.

In [3]:
print(f'Cantidad de datos duplicados: {df.duplicated().sum()}')
print(f'Porcentaje de datos duplicados: {round((df.duplicated().sum()/df.shape)[0] * 100, 2)} %')

Cantidad de datos duplicados: 3854
Porcentaje de datos duplicados: 3.85 %


El porcentaje de datos duplicado es muy pequeño por lo tanto se eliminan para no tener problemas en secciones posteriores.

In [4]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

A continutacion se consulta el numero de datos faltantes, siendo este cero para todas las columnas del dataset.

In [5]:
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

## Definicion de las caracteristicas y etiquetas

----------------------

En primer lugar, se separan las caracteristicas y las etiquetas, asignadoles la variable X e y respectivamente. Nuestra etiqueta sera la columna "Diabetes". Tambien se comprueban la dimension de los arreglos resultantes.

In [6]:
X = df.drop(['diabetes'], axis = 1)
y = df['diabetes']
print(f'Dimensiones caracteristicas: {X.shape}')
print(f'Dimensiones etiquetas: {y.shape}')

Dimensiones caracteristicas: (96146, 8)
Dimensiones etiquetas: (96146,)


Dado que tenemos 3 tipos de variables que son, boleanas, númericas y categóricas. Se separan segun su tipo para aplicarle metodos de preprocesamiento segun su conveniencia. Por lo tanto los grupos quedaran de la siguiente manera.

* Variables boleanas : hypertension, heart_disease
* Variables númericas: age, bmi, HbA1c_level, blood_glucose_level
* Variables categóricas: gender, smoking_history

Para terminar, se verifica sus dimensiones.

In [7]:
bol = ['hypertension', 'heart_disease']
num = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
cat = ['gender', 'smoking_history']

X_bol = X[bol].values
X_num = X[num].values
X_cat = X[cat].values

print (f'Dimensiones variables boleanas: {X_bol.shape}')
print(f'Dimensiones variables numericas: {X_num.shape}')
print(f'Dimensiones varaibles categoricas: {X_cat.shape}')

Dimensiones variables boleanas: (96146, 2)
Dimensiones variables numericas: (96146, 4)
Dimensiones varaibles categoricas: (96146, 2)


Tambien se transforma a un arreglo las etiquetas.

In [8]:
y =  y.values
print(f'Dimensiones etiquetas: {y.shape}')

Dimensiones etiquetas: (96146,)


Se observan los arreglos obtenidos para cada tipo de variable.

In [9]:
ind = [6, 100, 5203, 96117]

for i in ind:
  print(f'Ejemplo {i}:')
  print('Variables boleanas:', X_bol[i])
  print('Variables numericas:', X_num[i])
  print('Variables categoricas:', X_cat[i])
  print('Etiqueta:', y[i])
  print()

Ejemplo 6:
Variables boleanas: [0 0]
Variables numericas: [ 44.    19.31   6.5  200.  ]
Variables categoricas: ['Female' 'never']
Etiqueta: 1

Ejemplo 100:
Variables boleanas: [0 0]
Variables numericas: [ 38.    27.32   6.   158.  ]
Variables categoricas: ['Male' 'never']
Etiqueta: 0

Ejemplo 5203:
Variables boleanas: [0 0]
Variables numericas: [ 51.    27.32   3.5  200.  ]
Variables categoricas: ['Female' 'No Info']
Etiqueta: 0

Ejemplo 96117:
Variables boleanas: [1 0]
Variables numericas: [ 51.    28.67   6.1  145.  ]
Variables categoricas: ['Female' 'No Info']
Etiqueta: 0



### Normalización variables númericas

--------------------------------
Luego de analizar los resultados obtenidos en la etapa de entendimiento del negocio, se llego a la conclusión de que el metodo apropiado para preprocesar la data era la normalización ya que segun las distribuciones de probabilidad los rangos de los valores eran muy lejanos entre si en la mayoria de casos. Para esto utilzamos la herramienta MinMaxScaler de sickit-learn.

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))  
X_num_minmax = scaler.fit_transform(X_num) 

Se observan las transformaciones de los datos luego de aplicar la normalización.

In [11]:
for i in range(3):
  print('Ejemplo:', i)
  print('Original: ', X_num[i])
  print('MinMax: ', X_num_minmax[i])
  print()

Ejemplo: 0
Original:  [ 80.    25.19   6.6  140.  ]
MinMax:  [1.         0.17717087 0.56363636 0.27272727]

Ejemplo: 1
Original:  [54.   27.32  6.6  80.  ]
MinMax:  [0.67467467 0.20203081 0.56363636 0.        ]

Ejemplo: 2
Original:  [ 28.    27.32   5.7  158.  ]
MinMax:  [0.34934935 0.20203081 0.4        0.35454545]



### Codificación variables categoricas

-----------------------------

Dado que tenemos dos variables categoricas, se aplicara el metodo de one-hot encoding para que estas puedan ser utilizadas por un modelo machine learning y poder brindar informacion adicional y mejorar el rendimiento del modelo. Primero se verifica si el numero de asignaciones por variable es viable para aplicar one hot encoding.

In [12]:
for var in cat:
  print(f'Valores posibles de {var}: \t{X[var].nunique()}')

Valores posibles de gender: 	3
Valores posibles de smoking_history: 	6


Una vez verificado se procede a aplicar OneHotEncoder de scikit-learn, para obtener una transformación.

In [13]:
enc = OneHotEncoder(sparse_output=False)     # Declaramos el Transformer "OneHotEncoder".
X_cat_onehot = enc.fit_transform(X_cat) # Usamos "fit_transform" para obtener la matriz transformada.

Se observan los cambios realizados con one hot encoding a las varaibles categoricas.

In [14]:
ids = [0, 1, 15, 20]
for i in ids:
  print('Ejemplo:', i)
  print('Original: ', X_cat[i])
  print('One Hot: ', X_cat_onehot[i])
  print()

Ejemplo: 0
Original:  ['Female' 'never']
One Hot:  [1. 0. 0. 0. 0. 0. 0. 1. 0.]

Ejemplo: 1
Original:  ['Female' 'No Info']
One Hot:  [1. 0. 0. 1. 0. 0. 0. 0. 0.]

Ejemplo: 15
Original:  ['Male' 'No Info']
One Hot:  [0. 1. 0. 1. 0. 0. 0. 0. 0.]

Ejemplo: 20
Original:  ['Male' 'current']
One Hot:  [0. 1. 0. 0. 1. 0. 0. 0. 0.]



Para terminar esta subseccion se concatenan los arreglos con las transformaciones obtenidas, cabe resaltar que para el caso de las variables boleanas no se realizo ninguna transformación ya que no necesitan alguna.

In [15]:
X_fin = np.concatenate((X_num_minmax, X_bol ,X_cat_onehot),axis=1) 
print(X_fin.shape)

(96146, 15)


## Separación del data set en entrenamiento y prueba.

--------------

Se separa el set de datos en 80% entrenamiento y 20% prueba. Con el fin de prepararlo para ingresar a un modelo para ser entrenado. El conjunto de validación no fue extraido ya que se utilizaran metodos de validacion cruzada mas adelante y por lo tanto sera configurado en ese momento.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

Finalmente, se obtienen las dimensiones de los datos de entrenamiento y evaluación para las caracteristicas y la clase.

In [26]:
print(f'Dimensiones X_train: {X_train.shape}')
print(f'Dimensiones X_test: {X_test.shape}')
print(f'Dimensiones y_train: {y_train.shape}')
print(f'Dimensiones y_test: {y_test.shape}')

Dimensiones X_train: (76916, 15)
Dimensiones X_test: (19230, 15)
Dimensiones y_train: (76916,)
Dimensiones y_test: (19230,)


Se almacenan los arreglos numpy.

In [27]:
np.savez('train_test_array.npz', X_train = X_train, X_test=X_test, y_train=y_train, y_test=y_test)